In [33]:
import pandas as pd
import numpy as np
df= pd.read_csv('play_tennis.csv')
df.drop('day',axis=1,inplace=True)
df

,outlook,temp,humidity,wind,play
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes
5,Rain,Cool,Normal,Strong,No
6,Overcast,Cool,Normal,Strong,Yes
7,Sunny,Mild,High,Weak,No
8,Sunny,Cool,Normal,Weak,Yes
9,Rain,Mild,Normal,Weak,Yes


In [8]:
# from sklearn.preprocessing import LabelEncoder
# Le = LabelEncoder()
# df['outlook'] = Le.fit_transform(df['outlook'])
# df['humidity'] = Le.fit_transform(df['humidity'])
# df['wind'] = Le.fit_transform(df['wind'])
# df['play'] = Le.fit_transform(df['play'])
# df

,outlook,temp,humidity,wind,play
0,2,Hot,0,1,0
1,2,Hot,0,0,0
2,0,Hot,0,1,1
3,1,Mild,0,1,1
4,1,Cool,1,1,1
5,1,Cool,1,0,0
6,0,Cool,1,0,1
7,2,Mild,0,1,0
8,2,Cool,1,1,1
9,1,Mild,1,1,1


In [11]:
# Entropy of Data H(D) i.e class 
def EntropyD(data):  
    unique , counts = np.unique(data , return_counts=True)
    ans = 0
    for i in counts: # applying entropy ka formula pilog2(pi) 
        ans += i/(len(data)) * np.log2(i/(len(data)))
    
    ans = (-1)*ans
    return ans

In [43]:
def split_numerical(ns,classes):
    n = np.argsort(ns)
    Unique = np.sort(np.unique(ns))
    # sorting classes according to temp
    c = []
    A = []
    for i in n:
        c.append(classes[i])
        A.append(ns[i])
            
    z = np.zeros((len(Unique),2)) # zeroes matrix
    cz = np.zeros(2)  # zeroes matrix size of unique classes
    for i in range(len(Unique)):
        for j in range(len(A)):
            
            # cz[0] will have yes and cz[1] will have no
            if c[j] == 'Yes':
                cz[0] = cz[0] + 1
            if c[j] == 'No':
                cz[1] = cz[1] + 1
            
            
            if Unique[i] <= A[j]:
                
                # equals the total number of classes
                z[i][0] = cz[0] 
                z[i][1] = cz[1]

                break    
    left = z
    right = z[len(Unique)-1] - z
    
    
    return left,right,Unique


def Entropy(X,Y):
    entropy = ( ((X)*np.log2(X)) + ((Y)*np.log2(Y)) )
                   
    return -1*entropy

def finding_left_right_Entropies(left,right,splits):
    m,n = left.shape
    print(m,n)
    entropies = {}
    print(left)
    for i in range(m-1):
        rsum = sum(right[i])
        lsum = sum(left[i])
        loggy_right = 0
        loggy_left  = 0
        
            
        e_left = Entropy(left[i][0]/lsum,left[i][1]/lsum)
        e_right = Entropy(right[i][0]/rsum,right[i][1]/rsum)
        
        splitEntropy = e_left + e_right
        
        entropies[splits[i]] = splitEntropy
         
        
    print(entropies)

l, r,s =  split_numerical(df['temp'],df['play'])
finding_left_right_Entropies(l,r,s)

3 2
[[ 1.  0.]
 [ 4.  2.]
 [10.  5.]]
{'Cool': nan, 'Hot': 1.8365916681089791}


<ipython-input-43-7ee32159ac8b>:38: RuntimeWarning: divide by zero encountered in log2
  entropy = ( ((X)*np.log2(X)) + ((Y)*np.log2(Y)) )
<ipython-input-43-7ee32159ac8b>:38: RuntimeWarning: invalid value encountered in double_scalars
  entropy = ( ((X)*np.log2(X)) + ((Y)*np.log2(Y)) )


In [44]:
# Entropy of each column w.r.t class 
def entropy(feature,label):
    
    unique_label , counts_label = np.unique(label , return_counts=True)
    unique_feature , counts_feature = np.unique(feature , return_counts=True)

    yes = []
    no  = []
    yes_c = 0
    no_c  = 0
    for j in unique_feature: 
        yes_c = 0
        no_c  = 0 
        for i in range(len(feature)):
            if j == feature[i]:
                if  label[i] == 'Yes':
                    yes_c +=1
                else:
                    no_c+=1

        yes.append(yes_c)
        no.append(no_c)
        
    entropyList=[]
    for i in range(0,len(unique_feature)): # applying entropy ka formula pilog2(pi) 
        ans=0
        if (yes[i]==0):
            ans += (no[i]/(counts_feature[i])*np.log2(no[i]/(counts_feature[i])))
        elif (no[i]==0):
            ans += (yes[i]/(counts_feature[i])*np.log2(yes[i]/(counts_feature[i])))      
        else:
            ans += (yes[i]/(counts_feature[i])*np.log2(yes[i]/(counts_feature[i])))
            ans += (no[i]/(counts_feature[i])*np.log2(no[i]/(counts_feature[i])))
        ans = (-1)*ans
        entropyList.append(ans)
    
    return entropyList

In [26]:
def infoGain(feature, label):
    total = 0
    value , total = np.unique(feature , return_counts=True)
    size = len(value)
    labell = EntropyD(label)
    templist = entropy(feature, label)
    res = 0
    for i in range(0, size):
        temp = templist[i] * total[i]/len(label)
        res = res + temp 
    ig = labell - res
    ig = round(ig, 4)
    print("Information Gain:" , ig)
    return ig


In [27]:
# information gain of each feature 
ig1 = infoGain(np.array(df['outlook']) , np.array(df['play']))
ig2 = infoGain(np.array(df['temp']) , np.array(df['play']))
ig3 = infoGain(np.array(df['humidity']) , np.array(df['play']))
ig4 = infoGain(np.array(df['wind']) , np.array(df['play']))

Information Gain: 0.9403
Information Gain: 0.9403
Information Gain: 0.9403
Information Gain: 0.9403


In [15]:
data=[]
for i in df['outlook'].unique():  # storing the unique values of outlook in a list 
    data.append(df[df['outlook']==i])  

In [16]:
# splitting data on the basis of Outlook ki unique values
df1 = pd.DataFrame(data[0])
print(df1, "\n")
df2 = pd.DataFrame(data[1])
print(df2, "\n")
df3 = pd.DataFrame(data[2])
print(df3, "\n")

    outlook  temp  humidity  wind  play
0         2   Hot         0     1     0
1         2   Hot         0     0     0
7         2  Mild         0     1     0
8         2  Cool         1     1     1
10        2  Mild         1     0     1 

    outlook  temp  humidity  wind  play
2         0   Hot         0     1     1
6         0  Cool         1     0     1
11        0  Mild         0     0     1
12        0   Hot         1     1     1 

    outlook  temp  humidity  wind  play
3         1  Mild         0     1     1
4         1  Cool         1     1     1
5         1  Cool         1     0     0
9         1  Mild         1     1     1
13        1  Mild         0     0     0 



Overcast is pure so no need to further break it down

In [37]:
ig2 = infoGain(np.array(df1['temp']) , np.array(df1['play']))
ig3 = infoGain(np.array(df1['humidity']) , np.array(df1['play']))
ig4 = infoGain(np.array(df1['wind']) , np.array(df1['play']))

Information Gain: 0.971
Information Gain: 0.971
Information Gain: 0.971


In [38]:
# as humidity has highest info gain 
data=[]
for i in df1['humidity'].unique():
    print(i)
    data.append(df1[df1['humidity']==i]) 

0
1


In [39]:
df4 = pd.DataFrame(data[0])
print(df4, "\n")
df5 = pd.DataFrame(data[1])
print(df5, "\n")

   outlook  temp  humidity  wind  play
0        2   Hot         0     1     0
1        2   Hot         0     0     0
7        2  Mild         0     1     0 

    outlook  temp  humidity  wind  play
8         2  Cool         1     1     1
10        2  Mild         1     0     1 



No further splitting required as classes are pure

In [40]:
# When dataframe reduced w.r.t rain

ig2 = infoGain(np.array(df3['temp']) , np.array(df3['play']))
ig3 = infoGain(np.array(df3['humidity']) , np.array(df3['play']))
ig4 = infoGain(np.array(df3['wind']) , np.array(df3['play']))

Information Gain: 0.971
Information Gain: 0.971
Information Gain: 0.971


In [41]:
data=[]
for i in df3['wind'].unique():
    print(i)
    data.append(df3[df3['wind']==i]) 

1
0


In [42]:
df7 = pd.DataFrame(data[0])
print(df7, "\n")
df8 = pd.DataFrame(data[1])
print(df8, "\n")

   outlook  temp  humidity  wind  play
3        1  Mild         0     1     1
4        1  Cool         1     1     1
9        1  Mild         1     1     1 

    outlook  temp  humidity  wind  play
5         1  Cool         1     0     0
13        1  Mild         0     0     0 



No further splitting required as classes are pure